In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from IPython.display import Video
import plotly.express as px
import pandas as pd
import timeit

# TODO Could also try this technique with audio, wonder if blurring would work

In [ ]:
from sklearn.cluster import MeanShift, estimate_bandwidth, KMeans, DBSCAN
from sklearn.neighbors.kde import KernelDensity
from scipy.signal import argrelextrema

def kde_cluster(Xdf, kernel="gaussian", bandwidth=10, plot=False):
    """
    Uses KDE to create clusters out of word vecs
    """
    X = np.array(Xdf['x'])
    X = X.reshape(-1, 1)

    # TODO There is probably bias at the boundaries, should mirror X
    kde = KernelDensity(kernel=kernel, bandwidth=bandwidth).fit(X)
    s = np.linspace(0, np.max(X)*1.5)
    e = kde.score_samples(s.reshape(-1, 1))

    # Reshape back to a 1 by N array
    X = X.reshape(1, -1)

    minima = argrelextrema(e, np.less)[0]
    # Use the linspace to convert back into word indexes
    minima = [s[m] for m in minima]
    # (0, minima 1), (minima 1, minima 2), ... (minima n-1, minima n), (minima n, end)
    minima_pairs = list(zip(np.insert(minima, 0, 0), np.append(minima, s[-1])))

    clusters = [
      np.unique(X[np.logical_and(X >= m1, X < m2)]) for m1, m2 in minima_pairs
    ]

    if plot:
      plt.plot(s, e)
      plt.show()
      print(f"Number of clusters: {len(clusters)}")
      for c in clusters:
          print("\t", len(c), np.unique([int(x) for x in c]))

    return clusters

# Store frame numbers in the clusters instead of the average of the last frame
def frame_idxs_clusters(aavg_clusters, df):
  df_2 = df_avg.copy()
  df_2['cluster'] = np.full_like((len(averages)), -1)

  # clusters = []
  for i, c in enumerate(avg_clusters):
      # clusters.append([])
      for j, frame in enumerate(frames[:-1]):
          if normal_averages[j] >= np.min(c) and normal_averages[j] <= np.max(c):
              clusters[i].append(j)
              df_2['cluster'][j] = i
              continue
  
  return df_2

In [ ]:
# TODO How to estimate bandwidth for 2d data?
avg_clusters = kde_cluster(df, plot=True, bandwidth=5)
# TODO Actually returns a df now
clusters = frame_idxs_clusters(avg_clusters)

clusters = np.array(clusters)
clusters.shape, clusters[1][0]